In [1]:
import re
import pandas as pd
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.linear_model import LogisticRegressionCV
import numpy as np
import boto3

In [2]:
s3 = boto3.resource(service_name = 's3',
                    region_name = 'us-east-2',
                    aws_access_key_id = ' ',
                    aws_secret_access_key = '')

In [3]:
df = pd.read_csv('FullDataset.csv', encoding = 'ISO-8859-1')

In [12]:
s3.Bucket('testrini').upload_file(Filename = 'FullDataset.csv', Key = 'FullDataset.csv')

In [13]:
# print out bucket names
for bucket in s3.buckets.all():
  print(bucket.name)

testrini


In [14]:
for obj in s3.Bucket('testrini').objects.all():
  print(obj)

s3.ObjectSummary(bucket_name='testrini', key='FullDataset.csv')
s3.ObjectSummary(bucket_name='testrini', key='bar.csv')
s3.ObjectSummary(bucket_name='testrini', key='foo.csv')
s3.ObjectSummary(bucket_name='testrini', key='xgboost-as-a-built-in-algo/test/test.csv')
s3.ObjectSummary(bucket_name='testrini', key='xgboost-as-a-built-in-algo/train/train.csv')


In [16]:
#obj = s3.Bucket('testrini').Object('FullDataset.csv').get()
#temp = pd.read_csv(obj['Body'], index_col = 0,encoding='ISO-8859-1')

In [4]:
df

,Unnamed: 0,title,text,source,label,title_text,type,newstitle,publish_date,content
0,0,ÌÕItÌ¥s actually the safest time to fly.ÌÒ,NaN,NaN,0,actually safest time fly thousands flights...,article,"""Photos of deserted, nearly empty airports aro...",2020-03-01T18:39:00Z,thousands of flights around the world have bee...
1,1,"""Strengthening accountability to end TB""",NaN,NaN,1,strengthening accountability end tb today re...,article,"""WHO offers national assessment checklist for ...",NaN,who today released a checklist to enable membe...
2,2,Lack of Paid Sick Leave a Coronavirus Threat,NaN,NaN,1,lack paid sick leave coronavirus threat edit...,article,"""Lack of Paid Sick Leave a Coronavirus Threat""",NaN,editor 's note for the latest updates on the 2...
3,3,States are imposing mandatory leave and statew...,NaN,NaN,0,states imposing mandatory leave statewise sc...,post,NaN,NaN,NaN
4,4,Guidance for Administrators in Parks and Recre...,NaN,NaN,1,guidance administrators parks recreational ...,article,"""Information for Laboratories about Coronaviru...",2/11/20,these sequences are intended to be used for th...
...,...,...,...,...,...,...,...,...,...,...
5078,5078,ÛÏThe mainstream media pretended there was a ...,NaN,NaN,0,mainstream media pretended deadly surge c...,post,"""What the MSM Was Afraid to Tell You""",NaN,this week while the mainstream mainstream medi...
5079,5079,Repercussions From Antibiotics Overuse During ...,NaN,NaN,1,repercussions antibiotics overuse pandemic ...,article,"""Access denied | www.webmd.com used Cloudflare...",NaN,this website is using a security service to pr...
5080,5080,Study Confirms Cats Can Become Infected With a...,NaN,NaN,1,study confirms cats become infected may tra...,article,"""""",NaN,NaN
5081,5081,Texas Gov. Pauses Economic Reopening,NaN,NaN,1,texas gov pauses economic reopening website ...,article,"""Access denied | www.webmd.com used Cloudflare...",NaN,this website is using a security service to pr...


In [5]:
print(df.shape)
print(df['label'].value_counts())

(5083, 10)
1    3639
0    1444
Name: label, dtype: int64


In [6]:
def preprocessor(text):
  text = re.sub('<[^>]*>', '', text)
  text = re.sub(r'[^\w\s]', '', text)
  text = re.sub(r'[^\w]', ' ', text)
  text = re.sub(r'[\s+]', ' ', text)
  text = text.lower()
  return text
string = "The coronavirus, a man made virus, was created in China according to https://website.com/covid"
preprocessor(string)


'the coronavirus a man made virus was created in china according to httpswebsitecomcovid'

In [7]:
df['title_text'] = df['title_text'].apply(preprocessor)

In [8]:
a = 0
count = 0
largest = []
for i in range(len(df['title_text'])):
  a+= len(list(df['title_text'][i]))
  largest.append(len(list(df['title_text'][i])))
  if len(list(df['title_text'][i])) == 24414:
    string = (df['title_text'][i])
  count += 1
print('avg:',a/count)
print('max char len:', max(largest))
print('min char len:',min(largest))

avg: 921.5888254967539
max char len: 24414
min char len: 9


In [9]:
print(string)

pandemic   politics  survival  horizons   new type  dictatorship  breakdown   global liberal world order   foundations   happening    global breakdown   world order    matter   whether  nature   coronavirus  artificial      even  principal importance whether    artificial   deliberately released   world government    epidemic  begun     fact   main thing   trace   world government  reacted  itto clarify  world government   totality  global political  economic elites   intellectuals  media mediacrats  serve    world government necessarily exists    global scale   strictlydefined fundamental norms  determine  basic parameters  politics economics  ideologyin  economy   recognized norm  capitalism  market economy   disputed   north korea       important  china  presents   version  national state capitalism   management   communist partyin politics   recognized norm  parliamentary liberal democracy based  civil society   subject  source  legality  legitimacy besides north korea almost every

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
stop_words=set(stopwords.words('english'))
clean_stopword = re.compile(r"\b(" + "|".join(stop_words)+")\\W", re.I)
def remove_stopwords(sentence):
  return clean_stopword.sub(" ", sentence)
df['title_text'] = df['title_text'].apply(remove_stopwords)
string = "the coronavirus a man made virus was created in china according to httpswebsitecomcovid"
remove_stopwords(string)

' coronavirus  man made virus  created  china according  httpswebsitecomcovid'

In [12]:
string

'the coronavirus a man made virus was created in china according to httpswebsitecomcovid'

In [13]:
df['title_text']

0        actually  safest time  fly thousands  flights...
1       strengthening accountability  end tb  today re...
2       lack  paid sick leave  coronavirus threat edit...
3       states  imposing mandatory leave  statewise sc...
4       guidance  administrators  parks  recreational ...
                              ...                        
5078     mainstream media pretended    deadly surge  c...
5079    repercussions  antibiotics overuse  pandemic  ...
5080    study confirms cats  become infected   may tra...
5081    texas gov pauses economic reopening  website  ...
5082    colleges  universities  higher learning cdc co...
Name: title_text, Length: 5083, dtype: object

In [14]:
stemmer = PorterStemmer()
# def stemming(sentence):
#     stemSentence = ""
#     for word in sentence.split():
#         stem = stemmer.stem(word)
#         stemSentence += stem
#         stemSentence += " "
#     stemSentence = stemSentence.strip()
#     return stemSentence
def stemming(sentence):
  stemSentence = []
  for word in sentence.split():
    stem = stemmer.stem(word)
    stemSentence.append(stem)
  return stemSentence
stemming('coronavirus  man made virus  created  china according')
  


['coronaviru', 'man', 'made', 'viru', 'creat', 'china', 'accord']

In [15]:
df['title_text'] = df['title_text'].apply(stemming)

In [16]:
df['title_text']

0       [actual, safest, time, fli, thousand, flight, ...
1       [strengthen, account, end, tb, today, releas, ...
2       [lack, paid, sick, leav, coronaviru, threat, e...
3       [state, impos, mandatori, leav, statewis, scho...
4       [guidanc, administr, park, recreat, facil, seq...
                              ...                        
5078    [mainstream, media, pretend, deadli, surg, cov...
5079    [repercuss, antibiot, overus, pandem, websit, ...
5080    [studi, confirm, cat, becom, infect, may, tran...
5081    [texa, gov, paus, econom, reopen, websit, use,...
5082    [colleg, univers, higher, learn, cdc, commun, ...
Name: title_text, Length: 5083, dtype: object

In [31]:
len(df['title_text'])

5083

In [21]:
!pip3 install -U gensim
import gensim
from gensim.models import KeyedVectors

In [22]:
EMBEDDING_DIM=100
MIN_COUNT=4
WINDOW=5 # the max distance btw a target word and words around it.
WORKERS=3 #the num of partitions during training

In [23]:
title_text = df['title_text'].tolist()
type(title_text)

list

In [24]:
if 'carrier' in title_text:
  print(True)

In [25]:

model = gensim.models.Word2Vec(sentences=title_text, size=EMBEDDING_DIM,
                window = WINDOW, min_count = MIN_COUNT, workers=WORKERS)
words=list(model.wv.vocab)
print('Vocab size:{}'.format(len(words)))

Vocab size:7860


In [26]:
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

In [27]:
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')
wv['children']

array([-6.02211177e-01,  1.11514755e-01, -1.02601610e-01,  2.49327302e-01,
        2.99291849e-01, -7.30343238e-02, -5.43575227e-01, -1.84723973e-01,
        1.08394337e+00, -1.41092324e+00, -7.67262697e-01,  8.72772515e-01,
        6.69275403e-01,  7.19345510e-02, -5.54503381e-01, -1.33677018e+00,
       -4.59968179e-01, -4.51719373e-01,  3.73016149e-01, -1.98763907e-01,
       -6.20344698e-01, -2.36982554e-01, -6.52078807e-01,  2.61517495e-01,
        1.25242436e+00,  2.01398134e-01, -5.63823581e-02,  8.49937260e-01,
        4.27332148e-02,  1.32794702e+00, -7.02852190e-01,  1.27538323e+00,
        1.00661147e+00,  8.43592584e-01, -1.51611408e-02,  9.25414085e-01,
        1.08837593e+00, -1.32310414e+00, -2.46851668e-01, -7.10209846e-01,
        3.00451398e-01,  8.02641153e-01, -2.94685483e-01,  4.81487930e-01,
        8.33078265e-01, -1.21527553e+00, -3.96139175e-01,  5.57752132e-01,
        3.60735267e-01, -1.59792268e+00, -2.56283581e-02,  9.47295785e-01,
       -3.21766883e-01, -

In [28]:
#!pip install --upgrade pip
#print("hey1")
#!pip uninstall tensorflow --y
#print("hey2")
!pip install tensorflow
print("hey")
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

hey


In [29]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(title_text)
sequences = tokenizer.texts_to_sequences(title_text)

In [30]:
word_index = tokenizer.word_index
print('{} unique tokens.'.format(len(word_index)))

29138 unique tokens.


In [31]:
des_pad = pad_sequences(sequences, maxlen=500)
des_pad.shape

(5083, 500)

In [32]:
num_words = len(word_index)+1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
num_words

29139

In [33]:
for word, i in word_index.items():
  if word in wv:
    embedding_vector = wv[word]
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector 

In [34]:
# verbose 0: silent, doesnt show anything
# verbose 1: shows you progress bad
# verbose 2: shows one line per epoch

# check it out on keras, if u dont know how many epochs to do with model checkpoint, u can save on the best loss its ever seen or best validation accuracy its ever seen

# saves a model or weights at some interval so the model can be loaded later to continue training from the state saved

from keras.callbacks import ModelCheckpoint

mc = ModelCheckpoint(
    filepath = "weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5", # what to put here?
    monitor = 'val_loss',
    verbose = 0,
    save_best_only = True, # least loss... as opposed to saving each model at the end of each epoch
    save_weights_only = True, #
    mode = 'min') # min bc we want to minimize loss. max is for val_acc


# stops training when a monitored metric has stopped improving

from keras.callbacks import EarlyStopping

es = EarlyStopping(
    monitor = 'val_loss', # monitor the loss
    min_delta = 0.01, # wait patience periods for it to do better by the min delta, if it does not then stops training
    patience = 2, # number of epochs with no improvement
    verbose = 1, 
    mode = 'min')

#from keras.callbacks import LearningRateScheduler

#lrs = LearningRateScheduler(lambda epoch: 1./epoch) # at the end of each epoch change the learning rate 


# reduce learning rate when a metric has stopped improving 

from keras.callbacks import ReduceLROnPlateau # hardly uses this

# models often benefit from reducing the learning rate once learning stagnates
rlrop = ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.1,
    patience = 2, # if model does not see improvement by epsilon after 2 epochs then reduce the learning rate
    verbose = 0,
    mode = 'min',
    epsilon = 0.0001,
    cooldown = 2, # wait 2 epochs before monitoring the metrics again
    min_lr = 0 # minimum learning rate
 )



ModuleNotFoundError: No module named 'keras'

In [35]:
# CNN 
from keras import layers

import keras
from keras.models import Sequential
from keras.layers import Embedding
from keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, Dropout
from keras.initializers import Constant

model2 = Sequential()
model2.add(Embedding(num_words, EMBEDDING_DIM,
          embeddings_initializer = Constant(embedding_matrix),
          trainable=False))
model2.add(layers.Conv2D(128, 5, activation='relu'))
model2.add(layers.GlobalMaxPooling1D())
model2.add(layers.Dense(10, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
print(model2.summary())

ModuleNotFoundError: No module named 'keras'

In [37]:
#LSTM
!pip install keras
import keras
from keras.models import Sequential
from keras.layers import Embedding
from keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, Dropout
from keras.initializers import Constant

model= keras.Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM,
          embeddings_initializer = Constant(embedding_matrix),
          trainable=False))
model.add(Dropout(0.3))
model.add(keras.layers.LSTM(256))
model.add(Dropout(.3))
model.add(keras.layers.Dense(1, activation = "sigmoid")) 
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         2913900   
_________________________________________________________________
dropout (Dropout)            (None, None, 100)         0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               365568    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 3,279,725
Trainable params: 365,825
Non-trainable params: 2,913,900
_________________________________________________________________
None


In [ ]:
# CNN 
from keras import layers

model2 = Sequential()
model2.add(Embedding(num_words, EMBEDDING_DIM,
          embeddings_initializer = Constant(embedding_matrix),
          trainable=False))
model2.add(layers.Conv1D(128, 5, activation='relu'))
# gloabl max pooling has no hyperparameters to learn
model2.add(layers.GlobalMaxPooling1D())
model2.add(layers.Dense(10, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
print(model2.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 100)         2913900   
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 2,979,329
Trainable params: 65,429
Non-trainable params: 2,913,900
_________________________________________________________________
None


In [38]:
l = df['label'].tolist()

In [39]:
label_test = df['label'].tolist()
print(len(label_test))
print(df['label'].value_counts())

5083
1    3639
0    1444
Name: label, dtype: int64


In [40]:
VALIDATION_SPLIT=0.3

indices = np.arange(des_pad.shape[0])
np.random.shuffle(indices)
des_pad = des_pad[indices]
l = np.array(l)
l = l[indices.astype(int)]
num_vali_samples = int(VALIDATION_SPLIT*des_pad.shape[0])

In [41]:
len(l)

5083

In [42]:
X_train_pad = des_pad[:-num_vali_samples]
y_train = l[:-num_vali_samples]
X_test_pad = des_pad[-num_vali_samples:]
y_test = l[-num_vali_samples:]

In [43]:
print('Shape of X_train_pad tensor: {}'.format(X_train_pad.shape))
print('Shape of y_train tensor: {}'.format(y_train.shape))
print('Shape of X_test_pad tensor: {}'.format(X_test_pad.shape))
print('Shape of y_test tensor: {}'.format(y_test.shape))

Shape of X_train_pad tensor: (3559, 500)
Shape of y_train tensor: (3559,)
Shape of X_test_pad tensor: (1524, 500)
Shape of y_test tensor: (1524,)


In [44]:
des_pad

array([[   0,    0,    0, ..., 1070,   65, 5367],
       [   0,    0,    0, ...,   67,   25,    3],
       [   0,    0,    0, ...,  540,   52,   50],
       ...,
       [   0,    0,    0, ...,    6,  480, 1508],
       [   0,    0,    0, ...,  507,  136,   60],
       [   0,    0,    0, ...,  440,  418,   70]], dtype=int32)

In [46]:
# LSTM

model.fit(X_train_pad, y_train, batch_size=256, epochs=10, 
          validation_data=(X_test_pad, y_test), verbose=1)

Epoch 1/10
14/14 [==============================] - 65s 5s/step - loss: 0.3560 - accuracy: 0.8379 - val_loss: 0.3543 - val_accuracy: 0.8327
Epoch 2/10
14/14 [==============================] - 63s 4s/step - loss: 0.3023 - accuracy: 0.8722 - val_loss: 0.3182 - val_accuracy: 0.8675
Epoch 3/10
14/14 [==============================] - 62s 4s/step - loss: 0.2658 - accuracy: 0.8876 - val_loss: 0.2909 - val_accuracy: 0.8701
Epoch 4/10
14/14 [==============================] - 64s 5s/step - loss: 0.2509 - accuracy: 0.8977 - val_loss: 0.2601 - val_accuracy: 0.8924
Epoch 5/10
14/14 [==============================] - 63s 5s/step - loss: 0.2315 - accuracy: 0.9067 - val_loss: 0.2529 - val_accuracy: 0.8970
Epoch 6/10
14/14 [==============================] - 63s 5s/step - loss: 0.2300 - accuracy: 0.9045 - val_loss: 0.2759 - val_accuracy: 0.8944
Epoch 7/10
14/14 [==============================] - 64s 5s/step - loss: 0.2180 - accuracy: 0.9168 - val_loss: 0.2590 - val_accuracy: 0.8970
Epoch 8/10
14/14 [==

In [ ]:
# CNN

model2.fit(X_train_pad, y_train,
                    epochs=50,
                    validation_data=(X_test_pad, y_test),
                    batch_size=512,
                    callbacks = [mc, es, rlrop])

Epoch 1/50
7/7 [==============================] - 11s 2s/step - loss: 0.4911 - accuracy: 0.7185 - val_loss: 0.3795 - val_accuracy: 0.8025
Epoch 2/50
7/7 [==============================] - 11s 2s/step - loss: 0.3236 - accuracy: 0.8530 - val_loss: 0.2888 - val_accuracy: 0.8701
Epoch 3/50
7/7 [==============================] - 12s 2s/step - loss: 0.2586 - accuracy: 0.8862 - val_loss: 0.2607 - val_accuracy: 0.8904
Epoch 4/50
7/7 [==============================] - 12s 2s/step - loss: 0.2221 - accuracy: 0.9146 - val_loss: 0.2411 - val_accuracy: 0.8996
Epoch 5/50
7/7 [==============================] - 12s 2s/step - loss: 0.1940 - accuracy: 0.9292 - val_loss: 0.2326 - val_accuracy: 0.9068
Epoch 6/50
7/7 [==============================] - 11s 2s/step - loss: 0.1717 - accuracy: 0.9387 - val_loss: 0.2227 - val_accuracy: 0.9101
Epoch 7/50
7/7 [==============================] - 11s 2s/step - loss: 0.1548 - accuracy: 0.9449 - val_loss: 0.2175 - val_accuracy: 0.9134
Epoch 8/50
7/7 [==================

In [ ]:
def pred_result(model):
  y_pred = model.predict(X_test_pad)
  y=y_pred
  y=np.where(y>0.5, 1, y)
  y=np.where(y<0.5, 0, y)
  return y

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# LSTM results

y = pred_result(model)
print("---Test Set Results---")
print("Accuracy with lSTM:{}".format(accuracy_score(y_test, y)))
print(classification_report(y_test, y))

---Test Set Results---
Accuracy with lSTM:0.9048556430446194
              precision    recall  f1-score   support

           0       0.87      0.78      0.82       425
           1       0.92      0.95      0.94      1099

    accuracy                           0.90      1524
   macro avg       0.89      0.87      0.88      1524
weighted avg       0.90      0.90      0.90      1524



In [ ]:
# CNN results
y2 = pred_result(model2)
print("---Test Set Results---")
print("Accuracy with CNN:{}".format(accuracy_score(y_test, y2)))
print(classification_report(y_test, y2))

---Test Set Results---
Accuracy with CNN:0.9153543307086615
              precision    recall  f1-score   support

           0       0.85      0.84      0.85       425
           1       0.94      0.94      0.94      1099

    accuracy                           0.92      1524
   macro avg       0.90      0.89      0.89      1524
weighted avg       0.92      0.92      0.92      1524



In [ ]:
import json
import tensorflow as tf

In [ ]:
model.save("LSTM.h5")

In [ ]:
model = tf.keras.models.load_model("LSTM.h5")

In [ ]:
model2.save("CNN.h5")